## Unité 6: Problèmes de satisfaction des contraintes


Un CSP ou un réseau de contraintes est un triplet (X, D, C) où

X ={X1,.., Xn}  est un ensemble fini de n variables

D ={D1,.., Dn}, domaines finis où Xi ∈ Di ;

C ={C1,.., Cm}, m contraintes sur les variables où chacune définit un prédicat, qui est une relation sur un sous-ensemble particulier de variables (X). 

Une contrainte sur un ensemble de variables restreint les valeurs que peuvent prendre simultanément ses variables. Commençons par explorer la classe de base générique Constraint que nous utiliserons pour modéliser les contraintes. C'est une classe abstraite qui dispose d'une méthode abstraite **satisfied**


In [2]:

from typing import Generic, TypeVar, Dict, List, Optional, Any
from abc import ABC, abstractmethod
import copy

V = TypeVar('V')  # variable type
D = TypeVar('D')  # domain type

Unassigned = "Unassigned_Var"



# Base class for all constraints
class Constraint(Generic[V, D], ABC):
    # The variables that the constraint is between
    def __init__(self, variables: List[V]) -> None:
        self.variables = variables
    

    # Must be overridden by subclasses
    @abstractmethod
    def satisfied(self, assignment: Dict[V, D]) -> bool:
        ...



Une classe dérivée de la classe de base Contraint qui enveloppe une fonction définissant la logique d'une contrainte donnée peut être ecrire comme suit:

In [4]:

class FunctionConstraint(Constraint[V, D]):
    """
    Constraint which wraps a function defining the constraint logic

    Examples:

    >>> fc1 = FunctionConstraint(lambda x, y: x == y, ["a", "b"])
    >>> fc1.satisfied({'a': 1, 'b': 1})
    True
    >>> fc1.satisfied({'a': 1, 'b': 2})
    False
    >>> fc2 = FunctionConstraint(lambda x, y, z : x + y == z, [1, 2, 3])
    >>> fc2.satisfied({1: 1, 2: 2, 3: 3})
    True
    >>> fc2.satisfied({1: 5, 2: 1, 3: 3})
    False
    """

    def __init__(self, func, variables: List[V]) -> None:
        """
        @param func: Function wrapped and queried for constraint logic
        @type  func: callable object
        @param variables: list of variables on which this constraint applies
        """
        super().__init__(variables)
        self._func = func

    def satisfied(self, assignments: Dict[V, D]) -> bool:
        parms = [assignments.get(x, Unassigned) for x in self.variables]
        missing = parms.count(Unassigned)
        if missing:
            return True
        return self._func(*parms)

    def __repr__(self):
        return 'FunctionConstraint on ' + str(self.variables) + ' defined by \n' + str(self._func )


#### Exemples d'utilisation de la classe FunctionConstraint 

In [5]:
def XeqY(a,b) : 
    return FunctionConstraint(lambda x, y: x==y, [a, b])
print("is XeqY('a','b') satisfied on {'a': 1, 'b': 2} ? => ", XeqY('a','b').satisfied({'a': 1, 'b': 2}))
 

is XeqY('a','b') satisfied on {'a': 1, 'b': 2} ? =>  False


In [6]:
def XplusYeqZ(a, b, c): 
    return FunctionConstraint(lambda x, y, z : x + y == z, [a, b, c])
print("is XplusYeqZ(1, 2, 3) is satisfied on {1:1,2:2,3:3}= ", XplusYeqZ(1, 2, 3).satisfied({1: 1, 2: 2, 3: 3}))

is XplusYeqZ(1, 2, 3) is satisfied on {1:1,2:2,3:3}=  True


### Exercice 1
Déclarer et tester la contrainte XneqY


In [8]:
def XneqY(a,b) : 
    return FunctionConstraint(lambda x, y: x!=y, [a, b])
print("is XneqY('a','b') satisfied on {'a': 1, 'b': 2} ? => ", XneqY('a','b').satisfied({'a': 1, 'b': 2}))

is XneqY('a','b') satisfied on {'a': 1, 'b': 2} ? =>  True


In [9]:
def absXminYneqConst(a,b,c):
    return FunctionConstraint(lambda x, y, z : abs(x - y) == z, [a, b, c])

In [11]:
print("is XplusYeqZ(1, 2, 1) is satisfied on {1:1,2:2,3:3}= ", absXminYneqConst(1, 2, 1).satisfied({1: 1, 2: 2}))

is XplusYeqZ(1, 2, 1) is satisfied on {1:1,2:2,3:3}=  True


#### La contrainte tous différents 
c'est une contrainte spécialisée qui oblige chaque variable de décision dans un groupe donné à prendre une valeur différente de la valeur de chaque autre variable de décision dans ce groupe.

In [13]:

class AllDifferentConstraint(Constraint[V, D]):
    """
    Constraint enforcing that values of all given variables are different
    Example:

    >>>alldiff = AllDifferentConstraint(["a", "b", "c"])
    >>>alldiff.satisfied({1: 1, 2: 2, 3: 3})
    True
    >>> fc2.satisfied({1: 5, 2: 1, 3: 3})
    False
    """

    def __init__(self, variables: List[V]):
        super().__init__(variables)

    def satisfied(self, assignment: Dict[V, D]) -> bool:
        assigned = [x for x in self.variables if x in assignment.keys()]
        for i in range(len(assigned) - 1):
            for j in range(i + 1, len(assigned)):
                if assignment[assigned[i]] == assignment[assigned[j]]:
                    return False
        return True

    def __repr__(self):
        return 'AllDiff(' + str(self.variables) + ')'

### Exercice 2
Tester la contrainte AllDiff. Utiliser l'exemple donné dans la doc du class AllDifferentConstraint.

In [19]:
alldiff = AllDifferentConstraint(["a", "b", "c"])
alldiff.satisfied({'a': 1, 'b': 1, 'c': 3})

False

#### Modèle CSP
La première étape de la résolution d'un problème de satisfaction de contraintes consiste à modéliser le problème. Le modèle est composé de variables de décision et de contraintes. La classe CSP suivante vous permet de creer des modèles CSP.

domains: Dict[V, List[D]] indiquer que les domaines sont stockés dans un dictionnaire de la forme {var:liste-de-domaine} où liste-de-domaine est une pile de domaines de var dont chaque element di de la pile correspond au domaine de var au niveau i de l'arbre de recherche. curr_domains(var) est la fonction qui retourne le domaine courant de var. 

In [21]:
# A constraint satisfaction problem consists of variables of type V
# that have ranges of values known as domains of type D and constraints
# that determine whether a particular variable's domain selection is valid
class CSP(Generic[V, D]):
    def __init__(self, variables: List[V] = None, domains: Dict[V, List[D]] = None) -> None:
        self.variables: List[V] = variables  # variables to be constrained
        self.domains: Dict[V, List[List[D]]] = {}  # domain of each variable
        if domains:
            for v in domains.keys():
                self.domains[v] = [domains.get(v)]
        self.constraints: Dict[V, List[Constraint[V, D]]] = {}
        if variables:
            for variable in self.variables:
                self.constraints[variable] = []
                if variable not in self.domains:
                    raise LookupError("Every variable should have a domain assigned to it.")

    def add_variable(self, var: V, domain: D):
        if var in self.variables:
            msg = "Tried to insert duplicated variable %s" % repr(var)
            raise LookupError(msg)
        if not domain:
            raise ValueError("Domain is empty")
        self.variables.append(var)
        self.domains[var] = [domain]

    def add_constraint(self, constraint: Constraint[V, D]) -> None:
        for variable in constraint.variables:
            if variable not in self.variables:
                raise LookupError("Variable in constraint not in CSP")
            else:
                self.constraints[variable].append(constraint)

    # Check if the value assignment is consistent by checking all constraints
    # for the given variable against it
    def consistent(self, variable: V, assignment: Dict[V, D]) -> bool:
        for constraint in self.constraints[variable]:
            if not constraint.satisfied(assignment):
                return False
        return True
    
    def curr_domains(self, v):
        return self.domains[v][-1]

    def __repr__(self):
        rep = ['Variables:\n']
        the_constraints = []
        for v in self.variables:
            rep.append(str(v) + ' : ' + str(self.curr_domains(v)) + '\n')
            vc = self.constraints.get(v, [])
            the_constraints.extend([c for c in vc if c not in the_constraints])

        rep.append('Constraints:\n')
        for c in the_constraints:
            rep.append(repr(c) + '\n')
        return ''.join(rep)

    


#### Teste de la classe Problem

In [22]:
 
p = CSP(['a', 'b', 'c'], {'a': [0, 1], 'b': [0, 1], 'c': [0, 1]})
p.add_constraint(XeqY('a', 'b'))# we add the constraint a = b
p.add_constraint(XneqY('b', 'c'))# we add the constraint b != b
print(p)
print(p.consistent('a', {'a':0} ))
print(p.consistent('b', {'a':0, 'b':0} ))
print(p.consistent('c', {'a':0, 'b':0, 'c':0} ))
print(p.consistent('c', {'a':0, 'b':0, 'c':1} ))
print("the solution is: ", {'a':0, 'b':0, 'c':1} )

Variables:
a : [0, 1]
b : [0, 1]
c : [0, 1]
Constraints:
FunctionConstraint on ['a', 'b'] defined by 
<function XeqY.<locals>.<lambda> at 0x0000023629A777F0>
FunctionConstraint on ['b', 'c'] defined by 
<function XneqY.<locals>.<lambda> at 0x0000023629A75900>

True
True
False
True
the solution is:  {'a': 0, 'b': 0, 'c': 1}


#  Le puzzle crypto-arithmétique (The Crypto-Arithmetic Puzzle)

Dans cette partie, nous formulons le puzzle crypto-arithmétique présenté dans l'image ci-dessous comme un problème de satisfaction de contraintes.
<img src="send-more-money.png">

L'idée est que les lettres
"$\texttt{S}$", "$\texttt{E}$", "$\texttt{N}$", "$\texttt{D}$", "$\texttt{M}$", "$\texttt{O}$", "$\texttt{R}$", "$\texttt{Y}$" apparaissant dans ce puzzle
sont interprétées comme des variables s'étendant sur l'ensemble des chiffres décimaux, c'est-à-dire que ces variables peuvent prendre des valeurs dans
l'ensemble $\{0,1,2,3,4,5,6,7,8,9\}$. Ensuite, la chaîne "$\texttt{SEND}$" est interprétée comme un nombre décimal,
c'est-à-dire qu'il est interprété comme le nombre
$$\texttt{S} \cdot 10^3 + \texttt{E} \cdot 10^2 + \texttt{N} \cdot 10^1 + \texttt{D} \cdot 10^0.$$
Les chaînes "$\texttt{MORE}$ et "$\texttt{MONEY}$" sont interprétées de la même manière. Pour résoudre le problème
intéressant, l'hypothèse est que *différentes variables ont différentes* valeurs.
De plus, les chiffres au début d'un nombre doivent être *différents de $0$*. Ensuite, il faut trouver des valeurs telles que la formule
$$ (\texttt{S} \cdot 10^3 + \texttt{E} \cdot 10^2 + \texttt{N} \cdot 10 + \texttt{D})
  + (\texttt{M} \cdot 10^3 + \texttt{O} \cdot 10^2 + \texttt{R} \cdot 10 + \texttt{E})
  = \texttt{M} \cdot 10^4 + \texttt{O} \cdot 10^3 + \texttt{N} \cdot 10^2 + \texttt{E} \cdot 10 + \texttt{Y}.
$$
est vrai. 


Ajouter une ligne pour corriger la methode send_more_mony() suivate:


In [23]:
def send_more_mony():
    """ SEND+MORE=MONEY is a cryptarithmetic puzzle, meaning that it is about finding
    digits that replace letters to make a mathematical statement true.
    Each letter in the problem represents one digit (0–9).
    No two letters can represent the same digit. When a letter repeats,
    it means a digit repeats in the solution.
    To solve this puzzle by hand, it helps to line up the words.
         SEND
        +MORE
       =MONEY"""
    variables: List[str] = list(set('SENDMOREMONEY'))
    domains: Dict[str, List[int]] = {}
    for letter in variables:
        domains[letter] = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

    send_more_mony_problem: CSP[str, int] = CSP(variables, domains)
    func = lambda s, e, n, d, m, o, r, y: \
        s * 1000 + e * 100 + n * 10 + d \
        + m * 1000 + o * 100 + r * 10 + e \
        == m * 10000 + o * 1000 + n * 100 + e * 10 + y
    send_more_mony_problem.add_constraint(FunctionConstraint(func, variables))
    neq0 = lambda x: x != 0
    send_more_mony_problem.add_constraint(FunctionConstraint(neq0, ['S']))
    send_more_mony_problem.add_constraint(FunctionConstraint(neq0, ['M']))
    return send_more_mony_problem


Discutons la contrainte:

$$ (\texttt{S} \cdot 10^3 + \texttt{E} \cdot 10^2 + \texttt{N} \cdot 10 + \texttt{D})
  + (\texttt{M} \cdot 10^3 + \texttt{O} \cdot 10^2 + \texttt{R} \cdot 10 + \texttt{E})
  = \texttt{M} \cdot 10^4 + \texttt{O} \cdot 10^3 + \texttt{N} \cdot 10^2 + \texttt{E} \cdot 10 + \texttt{Y}.
$$

Le problème avec cette contrainte est qu'elle implique beaucoup trop de variables. Comme cette contrainte ne peut être

vérifiée lorsque toutes les variables ont des valeurs qui leur sont assignées, la recherche de retour en arrière serait essentiellement
se résument à une simple recherche par force brute. Nous aurions 8 variables et nous devions donc tester $10^{8}$
affectations possibles. Afin de faire mieux, nous devons effectuer l'addition dans la figure ci-dessus
colonne par colonne, comme on l'enseigne à l'école primaire. Pour pouvoir faire cela, nous devons introduire des <em>chiffres résidus</em> "$\texttt{C1}$", "$\texttt{C2}$", "$\texttt{C3}$" où $\texttt{C1}$ est le résidu produit en ajoutant
$\texttt{D}$ et $\texttt{E}$, $\texttt{C2}$ est le résidu produit en ajoutant
$\texttt{N}$, $\texttt{R}$ et $\texttt{C1}$, et $\texttt{C3}$ est le résidu produit en ajoutant
$\texttt{E}$, $\texttt{O}$ et $\texttt{C2}$.
 

                     (D + E)      % 10 == Y,  (D + E)      // 10 == C1,
                     (N + R + C1) % 10 == E,  (N + R + C1) // 10 == C2,
                     (E + O + C2) % 10 == N,  (E + O + C2) // 10 == C3,
                     (S + M + C3) % 10 == O,  (S + M + C3) // 10 == M
                     
réecrire le modéle CSP de ce problème en fonction de ces dernières contraintes.

In [24]:
def sum_mod10_eqX(vars):
    def sum_mod10_eqX_func(*args):
        l = list(args)
        return sum(l[:1])%10 == l[-1]
    return FunctionConstraint(sum_mod10_eqX_func,  vars)


def sum_div10_eqX(vars):
    def sum_div10_eqX_func(*args):
        l = list(args)
        return sum(l[:1])//10 == l[-1]
    return FunctionConstraint(sum_div10_eqX_func,  vars)


def crypto_csp():
    S, E, N, D, M, O, R, Y = 'S', 'E', 'N', 'D', 'M', 'O', 'R', 'Y'
    C1, C2, C3 = 'C1', 'C2', 'C3'
    
    digits       = [S, E, N, D, M, O, R, Y  ] 
    variables    = digits + [ C1, C2, C3]
    domain       = [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ]
    domains = {v:domain for v in variables}
    csp: CSP[str, int] = CSP(variables, domains)
    
    alldiff  = AllDifferentConstraint(digits)
    csp.add_constraint(alldiff)
    
    csp.add_constraint(sum_mod10_eqX([D, E, Y]))
    csp.add_constraint(sum_div10_eqX([D, E, C1]))
    
    csp.add_constraint(sum_mod10_eqX([N ,  R , C1, E]))
    csp.add_constraint(sum_div10_eqX([N ,  R , C1, C2]))
    
    
    csp.add_constraint(sum_mod10_eqX([E , O , C2, N]))
    csp.add_constraint(sum_div10_eqX([E , O , C2, C3]))
    
    csp.add_constraint(sum_mod10_eqX([S ,M, C3, O]))
    csp.add_constraint(sum_div10_eqX([S ,M, C3, M]))
     
    csp.add_constraint(FunctionConstraint(lambda x : x != 0, S))
    csp.add_constraint(FunctionConstraint(lambda x : x != 0, M))

 
    return csp

print(crypto_csp())

Variables:
S : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
E : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
N : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
D : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
M : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
O : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
R : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Y : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
C1 : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
C2 : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
C3 : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Constraints:
AllDiff(['S', 'E', 'N', 'D', 'M', 'O', 'R', 'Y'])
FunctionConstraint on ['S', 'M', 'C3', 'O'] defined by 
<function sum_mod10_eqX.<locals>.sum_mod10_eqX_func at 0x0000023629B04E50>
FunctionConstraint on ['S', 'M', 'C3', 'M'] defined by 
<function sum_div10_eqX.<locals>.sum_div10_eqX_func at 0x0000023629B04EE0>
FunctionConstraint on S defined by 
<function crypto_csp.<locals>.<lambda> at 0x0000023629B04F70>
FunctionConstraint on ['D', 'E', 'Y'] defined by 
<function sum_mod10_eqX.<locals>.sum_mod10_eqX_func at 0x0000023629B04B80>
FunctionConstraint on ['D', 'E', 'C1'] defined by 
<f

#### N-QUEENS

Le casse-tête des N-reines consiste à placer N reines d'échecs sur un échiquier NxN de manière à ce que deux reines ne se menacent pas l'une l'autre. Ici N est un nombre naturel. Comme le problème de coloration de graphe, NQueens est également implémenté dans le module csp. La classe NQueensCSP hérite de la classe CSP. Il apporte quelques modifications aux méthodes pour s'adapter à ce problème particulier. Les reines sont supposées être placées une par colonne, de gauche à droite. Cela signifie que la position (x, y) représente (var, val) dans le CSP. La contrainte qui doit être transmise au CSP est définie dans la fonction queens_constraint. La contrainte est satisfaite (vrai) si A, B sont vraiment la même variable, ou s'ils ne sont pas sur la même ligne, sur une diagonale décendante ou une diagonale accendante.

In [26]:
def queens(N):
    class queens_constraint(Constraint[int, int]):
        def __init__(self, columns: List[int]) -> None:
            super().__init__(columns)
            self.columns: List[int] = columns

        def satisfied(self, assignment: Dict[int, int]) -> bool:
            # q1c = queen 1 column, q1r = queen 1 row
            for q1c, q1r in assignment.items():
                # q2c = queen 2 column
                for q2c in range(q1c + 1, len(self.columns) + 1):
                    if q2c in assignment:
                        q2r: int = assignment[q2c] # q2r = queen 2 row
                        if q1r == q2r: # same row?
                            return False
                        if abs(q1r - q2r) == abs(q1c - q2c): # same diagonal?
                            return False
            return True # no conflict
        
        
    columns: List[int] = list(range(1, N+1))
    rows: Dict[int, List[int]] = {}
    for column in columns:
        rows[column] = list(range(1, N+1))
    csp: CSP[int, int] = CSP(columns, rows)  
    csp.add_constraint(queens_constraint(columns)) 
    return csp 

In [27]:
import  numpy as np
from PIL import Image
%matplotlib inline
import matplotlib.pyplot as plt
# Function to plot NQueensCSP from https://github.com/aimacode/aima-python/blob/master/notebook.py
def plot_queens(solution):
    n = len(solution) 
    board = np.array([2 * int((i + j) % 2) for j in range(n) for i in range(n)]).reshape((n, n))
    im = Image.open('queen_s.png')
    height = im.size[1]
    im = np.array(im).astype(np.float) / 255
    fig = plt.figure(figsize=(7, 7))
    ax = fig.add_subplot(111)
    ax.set_title('{} Queens'.format(n))
    plt.imshow(board, cmap='binary', interpolation='nearest')
    # NQueensCSP gives a solution as a dictionary
    for (k, v) in solution.items():
        newax = fig.add_axes([0.064 + ((k-1) * 0.112), 0.062 + ((n-1 - (v-1)) * 0.112), 0.1, 0.1], zorder=1)
        newax.imshow(im)
        newax.axis('off')
        # NQueensProblem gives a solution as a list
    fig.tight_layout()
    plt.show()                

#### Voisins d'une variable
Dans un CSP, les voisins d'une variable x sont l'ensemble constitué des variables partageant une contrainte avec x.

In [28]:
def vars_neighbors(csp: CSP, var:V):
    neighbors = set()
    if csp.constraints.get(var):
        for c in csp.constraints[var]:
            n = copy.deepcopy(c.variables)
            n.remove(var)
            neighbors.update(n)
    return neighbors

In [29]:
print('neighbors of a: ', vars_neighbors(p, 'a'))
print('neighbors of b: ', vars_neighbors(p, 'b'))

neighbors of a:  {'b'}
neighbors of b:  {'c', 'a'}


#### Opérations auxilières utiler lors de la résolution
Lors de la résolution d'un CSP, les opérations de base sont utilisées :
Affectez une variable pour étendre l'affectation actuelle. Annulez l'attribution d'une variable lors de la restauration. Maintenir les champs de variables.

In [30]:
def assign(csp, var, val, assignment):
    """Add {var: val} to assignment; Discard the old value if any.
    Do bookkeeping for the number of assignments (nassigns)."""
    if not hasattr(csp, 'nb_assigns'):
        csp.nb_assigns = 0
    csp.nb_assigns += 1
    assignment[var] = val
    csp.curr_domains(var).clear()
    csp.curr_domains(var).append(val)


def unassign(csp, var, assignment):
    """Remove {var: val} from assignment; that is backtrack.
    DO NOT call this if you are changing a variable to a new value;
    just call assign for that."""
    if var in assignment:
        del assignment[var]

def before_exdends_assignment (csp):
    """ Do bookkeeping for current domains """
    for v in csp.domains.keys():
        csp.domains[v].append(csp.curr_domains(v)[:])


def before_backtarck(csp, var, assignment):
    # Reset current domains to be the previous domains
    for v in csp.domains.keys():
        del csp.domains[v][-1]
    if var in assignment.keys():
        del assignment[var]

#### Algorithme de résolution de CSP backtracking_search recursive

In [31]:
def backtracking_search(csp: CSP, assignment=None) -> Optional[Dict[V, D]]:
    # assignment is complete if every variable is assigned (our base case)
    if assignment is None:
        assignment = {}
    if len(assignment) == len(csp.variables):
        return assignment
        # get all variables in the CSP but not in the assignment
    unassigned: List[V] = [v for v in csp.variables if v not in assignment]
    # get the every possible domain value of the first unassigned variable
    var: V = unassigned[0]
    before_exdends_assignment(csp)
    for value in csp.curr_domains(var)[:]:
        assign(csp, var, value, assignment)
        # if we're still consistent, we recurse (continue)
        if csp.consistent(var, assignment):
            result: Optional[Dict[V, D]] = backtracking_search(csp, assignment)
            # if we didn't find the result, we will end up backtracking
            if result is not None:
                return result

        unassign(csp, var, assignment)
    #we will backtrack to the previous variable assignement
    before_backtarck(csp, var, assignment)
    return None

In [32]:
p = CSP(['a', 'b', 'c'], {'a': [0, 1], 'b': [0, 1], 'c': [0, 1]})
p.add_constraint(FunctionConstraint(lambda  x, y : x==y, ['a', 'b']))
p.add_constraint(FunctionConstraint(lambda  x, y : x!=y, ['b', 'c']))

In [33]:
backtracking_search(p)

{'a': 0, 'b': 0, 'c': 1}

### Exercice 4
Résoudre le csp send_more_money en utilisant backtracking_search

In [34]:
csp = send_more_money()

NameError: name 'send_more_money' is not defined

Résoudre le csp crepto_csp en utilisant backtracking_search

### Exercice 5
Résoudre le probleme 8-Queen en utilisant backtracking_search

In [ ]:
# call the backtracking_search on queens problem and plot the solution
csp  = queens(8)

csp.plot_queens(backtracking_search(csp))

### Exercice 7


#### Machine qui retourne de la monnaie:

On s'intéresse à un distributeur automatique de boissons. L'utilisateur insère des pièces de monnaie pour un total de T centimes de Dirhams, puis il sélectionne une boisson, dont le prix est de P centimes de Dirhams (T et P étant des multiples de 10). Il s'agit alors de calculer la monnaie à rendre, sachant que le distributeur a en réserve Nb2 pièces de 2 Dh, Nb1 pièces de 1 Dh, Nb50 pièces de 50 centimes, Nb20 pièces de 20 centimes et Nb10 pièces de 10 centimes. 

#### Modèle CSP:
Variables : X = {X2, X1, X50, X20, X10} 
Les domaines spécifient que la quantité de pièces retournées, pour un type de pièce donné, est comprise entre 0 et le nombre de pièces de ce type que l'on a en réserve :  D(X2) = {0,1,...,Nb2} , D(X1) = {0,1,...,Nb1},  D(X50) = {0,1,...,Nb50} 
D(X20) = {0,1,...,Nb20}, et D(X10) = {0,1,...,Nb10} 
 Les contraintes spécifient que la somme à retourner doit être égale à la somme insérée moins le prix à payer :  
 
 C = { 200*X2 + 100*X1 + 50*X50 + 20*X20 + 10*X10 = T-P }
 
Ecrire la fonction vending_machine qui crée  et retourne un le CSP correspont à ce problème?


In [ ]:
def vending_machine():
    pass #Your code instead of pass

### Exercice 5
Résoudre le csp vending_machine obtenu en utilisant backtracking_search